In [26]:
from pyspark.sql import SparkSession
import os
import subprocess
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import VectorAssembler
# import pyspark.pandas as ps

In [2]:
spark = SparkSession.builder.master("local[4]").appName("Transform").config("spark.ui.port", '4050').getOrCreate()
sc = spark.sparkContext

24/12/18 20:52:12 WARN Utils: Your hostname, Jacobs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.104 instead (on interface en0)
24/12/18 20:52:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/18 20:52:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/18 20:52:13 WARN Utils: Service 'SparkUI' could not bind on port 4050. Attempting port 4051.


In [3]:
fileName = subprocess.check_output('ls extractedDataset | grep ".*.csv"', shell=True, text=True).removesuffix('\n')
path = os.getcwd() + "/extractedDataset/" + fileName
df = spark.read.csv(path, header=True, nullValue='')

In [4]:
print(df.show())

+-------------+------------+-----------------+--------------------+-----------------+-----------+--------------+-------------+--------+---------------+
|accessionYear|artistGender|artistNationality|             culture|       department|isHighlight|isTimelineWork|objectEndDate|objectID|     objectName|
+-------------+------------+-----------------+--------------------+-----------------+-----------+--------------+-------------+--------+---------------+
|         1979|        NULL|         American|                NULL|The American Wing|      false|         false|         1907|      14|           Coin|
|         1989|        NULL|         American|            American|The American Wing|      false|         false|         1814|     108|        Andiron|
|         1946|        NULL|             NULL|                NULL|The American Wing|      false|         false|         1890|     366|         Basket|
|         1946|        NULL|             NULL|            American|The American Wing|   

In [5]:
print(df.describe().show())

24/12/18 20:52:26 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------+-----------------+-----------------+--------------------+-----------+--------------+------------------+------------------+--------------------+
|summary|     accessionYear|artistGender|artistNationality|          culture|          department|isHighlight|isTimelineWork|     objectEndDate|          objectID|          objectName|
+-------+------------------+------------+-----------------+-----------------+--------------------+-----------+--------------+------------------+------------------+--------------------+
|  count|              5028|         128|             2671|             1690|                5048|       5048|          5048|              5048|              5048|                5027|
|   mean|1956.0318915686664|        NULL|             NULL|             NULL|                NULL|       NULL|          NULL|1411.4724643423137| 403327.7834786054|                NULL|
| stddev| 35.73250594529043|        NULL|             NULL|             NUL

In [6]:
#create custom function to convert gender column to boolean indicating if artist is female
# 0=Male, 1=Female
# in the database, they only store a value in the gender field if the artist is female
# otherwise it is null
def convertGender(s):
    if s == 'male':
        return 0
    else:
        return 1
# register user defined functions
convertGenderUDF = udf(lambda x:convertGender(x), IntegerType())

# take artistGender column, apply user-defined function to columns and put it in new column 
def genderColumn(df):
    # change nulls to male here because I don't know what datatype spark uses to represent nulls
    # So I cant detect if a value is null when I'm applying convertGender to each individual value in the column
    x = df.na.fill(value='male', subset=['artistGender'])
    # create new column that is made from applying the custom function to the gender column
    x = x.withColumn("isFemale", convertGenderUDF(col("artistGender")))
    x = x.drop(col("artistGender"))
    return x

In [7]:
# create dummy variables for department column
# meaning we add a column for each department, in each of these columns,
# the value will be 1 if that object is in that department, otherwise it is 0
# def departmentColumn(df):


In [8]:
def transformDataset(df):
    # in the database, they only store a value in the gender field if the artist is female
    # otherwise it is null
    # if there are multiple artists attributed to an object, the gender of each artist is shown, separated by a '|'
    x = genderColumn(df)
    # change the department column to a series of dummy variables
    # since there are a small number of distinct departments (X)
    return x

In [10]:
mod_df = transformDataset(df)
mod_df.show()

+-------------+-----------------+--------------------+-----------------+-----------+--------------+-------------+--------+---------------+--------+
|accessionYear|artistNationality|             culture|       department|isHighlight|isTimelineWork|objectEndDate|objectID|     objectName|isFemale|
+-------------+-----------------+--------------------+-----------------+-----------+--------------+-------------+--------+---------------+--------+
|         1979|         American|                NULL|The American Wing|      false|         false|         1907|      14|           Coin|       0|
|         1989|         American|            American|The American Wing|      false|         false|         1814|     108|        Andiron|       0|
|         1946|             NULL|                NULL|The American Wing|      false|         false|         1890|     366|         Basket|       0|
|         1946|             NULL|            American|The American Wing|      false|         false|         1870

In [24]:
columns = mod_df.columns
columns.remove('isHighlight')
print(columns)

['accessionYear', 'artistNationality', 'culture', 'department', 'isTimelineWork', 'objectEndDate', 'objectID', 'objectName', 'isFemale']


In [145]:
# drop culture because it has a ton of nulls, I think artist nationality along with the department it's from and year
#  it was made will still allow you to somewhat infer the culture it came from
no_nulls = mod_df.drop('culture')
# can't have null values in model
no_nulls = no_nulls.dropna()
no_nulls.count()

2649

In [146]:
from pyspark.ml.feature import Tokenizer
# change strings in nationality column to list of tokens, which is just each word in the string
nationalityTokenizer = Tokenizer(outputCol="nationalityWords")
nationalityTokenizer.setInputCol("artistNationality")
df2 = nationalityTokenizer.transform(no_nulls)

In [149]:
from pyspark.ml.feature import Word2Vec
# word2vec doesn't really extract any interesting patterns from the nationality column
# but it does help in reducing its dimensions
# in the bag of words approach, there were 108 dimensions
word2vec = Word2Vec(vectorSize=5, inputCol="nationalityWords", outputCol='nationalityVec', seed=123)
m2 = word2vec.fit(df2)
df3 = m2.transform(df2)
df3.show()

+-------------+-----------------+-----------------+-----------+--------------+-------------+--------+--------------------+--------+----------------+--------------------+
|accessionYear|artistNationality|       department|isHighlight|isTimelineWork|objectEndDate|objectID|          objectName|isFemale|nationalityWords|      nationalityVec|
+-------------+-----------------+-----------------+-----------+--------------+-------------+--------+--------------------+--------+----------------+--------------------+
|         1979|         American|The American Wing|      false|         false|         1907|      14|                Coin|       0|      [american]|[0.05077309161424...|
|         1989|         American|The American Wing|      false|         false|         1814|     108|             Andiron|       0|      [american]|[0.05077309161424...|
|         1946|         American|The American Wing|      false|         false|         1880|     779|                Bowl|       0|      [american]|[0

In [159]:
#change department to one hot encoding, since this is categorical
from pyspark.ml.feature import OneHotEncoder, StringIndexer
stringid = StringIndexer(inputCol='department', outputCol='departmentIndex')
m3 = stringid.fit(dataset=df3)
df4 = m3.transform(df3)

ohe = OneHotEncoder(inputCol='departmentIndex', outputCol='departmentCode')
m4 = ohe.fit(df4)
df5 = m4.transform(df4)
df5.show()

+-------------+-----------------+-----------------+-----------+--------------+-------------+--------+--------------------+--------+----------------+--------------------+---------------+--------------+
|accessionYear|artistNationality|       department|isHighlight|isTimelineWork|objectEndDate|objectID|          objectName|isFemale|nationalityWords|      nationalityVec|departmentIndex|departmentCode|
+-------------+-----------------+-----------------+-----------+--------------+-------------+--------+--------------------+--------+----------------+--------------------+---------------+--------------+
|         1979|         American|The American Wing|      false|         false|         1907|      14|                Coin|       0|      [american]|[0.05077309161424...|            5.0|(16,[5],[1.0])|
|         1989|         American|The American Wing|      false|         false|         1814|     108|             Andiron|       0|      [american]|[0.05077309161424...|            5.0|(16,[5],[1.

In [178]:
df5 = df5.drop('artistNationality')
df5 = df5.drop('department')
df5 = df5.drop('nationalityWords')
df5 = df5.drop('departmentIndex')
df5 = df5.drop('objectID')
df5.show()

+-------------+-----------+--------------+-------------+--------------------+--------+--------------------+--------------+
|accessionYear|isHighlight|isTimelineWork|objectEndDate|          objectName|isFemale|      nationalityVec|departmentCode|
+-------------+-----------+--------------+-------------+--------------------+--------+--------------------+--------------+
|         1979|      false|         false|         1907|                Coin|       0|[0.05077309161424...|(16,[5],[1.0])|
|         1989|      false|         false|         1814|             Andiron|       0|[0.05077309161424...|(16,[5],[1.0])|
|         1946|      false|         false|         1880|                Bowl|       0|[0.05077309161424...|(16,[5],[1.0])|
|         1950|      false|         false|         1889|             Drawing|       0|[0.05077309161424...|(16,[5],[1.0])|
|         1969|      false|         false|         1885|      Luncheon plate|       0|[0.05077309161424...|(16,[5],[1.0])|
|         1946| 

In [179]:
from pyspark.sql.functions import when
df6 = df5.withColumn('isHighlightInt', when(df5.isHighlight == True, 1).otherwise(0)).drop('isHighlight')
df6 = df6.withColumn('isTimelineWorkInt', when(df5.isTimelineWork == True, 1).otherwise(0)).drop('isTimelineWork')

In [180]:
# change strings in nationality column to list of tokens, which is just each word in the string
objectNameTokenizer = Tokenizer(outputCol="objectNameWords")
objectNameTokenizer.setInputCol("objectName")
df7 = objectNameTokenizer.transform(df6)

In [181]:
# using word2vec on objectName
word2vecObject = Word2Vec(vectorSize=5, inputCol="objectNameWords", outputCol='objectNameVec', seed=456)
m5 = word2vecObject.fit(df7)
df7 = m5.transform(df7)
df7.show()

+-------------+-------------+--------------------+--------+--------------------+--------------+--------------+-----------------+--------------------+--------------------+
|accessionYear|objectEndDate|          objectName|isFemale|      nationalityVec|departmentCode|isHighlightInt|isTimelineWorkInt|     objectNameWords|       objectNameVec|
+-------------+-------------+--------------------+--------+--------------------+--------------+--------------+-----------------+--------------------+--------------------+
|         1979|         1907|                Coin|       0|[0.05077309161424...|(16,[5],[1.0])|             0|                0|              [coin]|[0.0,0.0,0.0,0.0,...|
|         1989|         1814|             Andiron|       0|[0.05077309161424...|(16,[5],[1.0])|             0|                0|           [andiron]|[0.0,0.0,0.0,0.0,...|
|         1946|         1880|                Bowl|       0|[0.05077309161424...|(16,[5],[1.0])|             0|                0|              [bo

In [182]:
df8 = df7.drop('objectName')
df8 = df8.drop('objectNameWords')
df8.show()

+-------------+-------------+--------+--------------------+--------------+--------------+-----------------+--------------------+
|accessionYear|objectEndDate|isFemale|      nationalityVec|departmentCode|isHighlightInt|isTimelineWorkInt|       objectNameVec|
+-------------+-------------+--------+--------------------+--------------+--------------+-----------------+--------------------+
|         1979|         1907|       0|[0.05077309161424...|(16,[5],[1.0])|             0|                0|[0.0,0.0,0.0,0.0,...|
|         1989|         1814|       0|[0.05077309161424...|(16,[5],[1.0])|             0|                0|[0.0,0.0,0.0,0.0,...|
|         1946|         1880|       0|[0.05077309161424...|(16,[5],[1.0])|             0|                0|[-0.0416684746742...|
|         1950|         1889|       0|[0.05077309161424...|(16,[5],[1.0])|             0|                0|[0.53829401731491...|
|         1969|         1885|       0|[0.05077309161424...|(16,[5],[1.0])|             0|        

In [187]:
df8 = df8.withColumn('accessionYearInt', df8.accessionYear.cast(IntegerType())).drop('accessionYear')\
         .withColumn('objectDate', df8.objectEndDate.cast(IntegerType())).drop('objectEndDate')
df8.show()

+--------+--------------------+--------------+--------------+-----------------+--------------------+----------------+----------+
|isFemale|      nationalityVec|departmentCode|isHighlightInt|isTimelineWorkInt|       objectNameVec|accessionYearInt|objectDate|
+--------+--------------------+--------------+--------------+-----------------+--------------------+----------------+----------+
|       0|[0.05077309161424...|(16,[5],[1.0])|             0|                0|[0.0,0.0,0.0,0.0,...|            1979|      1907|
|       0|[0.05077309161424...|(16,[5],[1.0])|             0|                0|[0.0,0.0,0.0,0.0,...|            1989|      1814|
|       0|[0.05077309161424...|(16,[5],[1.0])|             0|                0|[-0.0416684746742...|            1946|      1880|
|       0|[0.05077309161424...|(16,[5],[1.0])|             0|                0|[0.53829401731491...|            1950|      1889|
|       0|[0.05077309161424...|(16,[5],[1.0])|             0|                0|[-0.0137707646936.

In [192]:
features = df8.columns
features.remove('isHighlightInt')
vector = VectorAssembler(inputCols=features, outputCol='features')
vec_train = vector.transform(df8)
vec_train.show()

+--------+--------------------+--------------+--------------+-----------------+--------------------+----------------+----------+--------------------+
|isFemale|      nationalityVec|departmentCode|isHighlightInt|isTimelineWorkInt|       objectNameVec|accessionYearInt|objectDate|            features|
+--------+--------------------+--------------+--------------+-----------------+--------------------+----------------+----------+--------------------+
|       0|[0.05077309161424...|(16,[5],[1.0])|             0|                0|[0.0,0.0,0.0,0.0,...|            1979|      1907|(30,[1,2,3,4,5,11...|
|       0|[0.05077309161424...|(16,[5],[1.0])|             0|                0|[0.0,0.0,0.0,0.0,...|            1989|      1814|(30,[1,2,3,4,5,11...|
|       0|[0.05077309161424...|(16,[5],[1.0])|             0|                0|[-0.0416684746742...|            1946|      1880|(30,[1,2,3,4,5,11...|
|       0|[0.05077309161424...|(16,[5],[1.0])|             0|                0|[0.53829401731491...|

In [ ]:
regularizationParam = 0.
from pyspark.ml.classification import LogisticRegression
logReg = LogisticRegression(labelCol='isHighlightInt')
logReg.setRegParam(regularizationParam)
logReg.setFeaturesCol('features')
logReg.setProbabilityCol('probability')
regModel = logReg.fit(vec_train)


In [225]:
summary = regModel.summary
summary.accuracy
summary.fMeasureByThreshold.show()

+--------------------+-------------------+
|           threshold|          F-Measure|
+--------------------+-------------------+
|  0.5006320358126095|                0.2|
| 0.42904976266737627|0.16666666666666666|
| 0.34437579956930786|0.14285714285714288|
|  0.2863215411637472|               0.25|
| 0.23990211818975604|0.33333333333333326|
| 0.15343274382417604|                0.4|
|  0.1492840344676416|0.36363636363636365|
| 0.13990995668508777| 0.3333333333333333|
| 0.13556598792045194| 0.3846153846153846|
| 0.13497756913321268| 0.4285714285714285|
| 0.11173487059727583|0.39999999999999997|
| 0.10321888582207406|              0.375|
| 0.06070314206155525| 0.3529411764705882|
| 0.05772800416464863| 0.3333333333333333|
|0.054508618267066655|0.31578947368421056|
|0.032817157892538584|                0.3|
|  0.0247140691803317| 0.2857142857142857|
|0.022397008527390794| 0.3181818181818182|
| 0.01676201907222752|0.30434782608695654|
|0.014699811399649265|0.29166666666666663|
+----------

In [170]:
spark.stop()